In [1]:
from pytubefix import YouTube
from pathlib import Path
import asyncio
import json
import ffmpeg
import logging

logging.basicConfig(filename='logs.log', encoding='utf-8', level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [2]:
links = json.load(Path('voginfo_links_parsed.json').open(encoding='utf-8'))
if Path('done.json').exists():
    done = json.load(Path('done.json').open(encoding='utf-8'))
else:
    done = []
    
path = 'D:/YandexDisk/SLR Project'

In [3]:
def clear_path():
    i = 0
    for fpath in Path(path).iterdir():
        if int(ffmpeg.probe(fpath)['format']['bit_rate']) < 5e5:
            fpath.unlink()
            i += 1
    for fpath in Path(path).iterdir():
        if fpath.stem.endswith(' (2)'):
            fpath.rename(fpath.with_stem(fpath.stem.rstrip(' (2)')))
            i += 1
    print(i)

In [4]:
def download_vid(link):
    vid = YouTube(link, use_po_token=True)
    
    path = 'D:/YandexDisk/SLR Project'
    downloaded = [f.stem for f in Path(path).iterdir()]
    
    try:
        if vid.title in downloaded:
            logger.info(f'NOTE video {link} is already downloaded')
            done.append(link)
        else:
            streams = vid.streams
            quality = streams.filter(res='720p') or streams.filter(res='480p') or streams.filter(res='1080p') \
                        or streams.filter(res='360p')
            target = quality[0].download(output_path=path)
            logger.info(f'SUCCESS video {link} was downloaded to {target}')
            done.append(link)
    except Exception as e:
        logger.error(f'with video {link}: {e}')
         # download_vid(link)

In [5]:
for link in links[::-1]:
    if link in done:
        continue
    download_vid(link)
    with open('done.json', 'w', encoding='utf-8') as f:
        json.dump(done, f, indent=4)